In [ ]:
import os
import glob
import numpy as np
import pandas as pd

# seed
np.random.seed(2020)
from keras.callbacks import Callback, EarlyStopping
from model.loss import myloss

from keras.optimizers import *
from keras.utils import multi_gpu_model
import sys
sys.path.append('../')
from dataload.my_image import MyImageDataGenerator
from model.coremodel import model_v9lc as my_model
from model.metric import count_accuracy
from keras.models import load_model
from model.DepthwiseConv2D import DepthwiseConv2D
print("import done")



In [ ]:
basedir = "/home/ubuntu/WorkSpace/DataSpace/shushu/data_spilit/v3/v3plan19_correct_data_link/"
basemodel_name = 'inception_v3'
model_image_size = (444, 592)
modelname = basemodel_name + '_v3plan19_correct_data_20181207'
#old_savemodel_path = './inception_v3_model_v9lc_plan6_20180401.h5'
old_savemodel_path = '../savemodel/1020v3plan18_v9lc_rescale3/inception_v3_model_v9lc_myscale3_v3plan18_20181020_0.0021_0.9944_wudi.h5'
savemodel_path = "../savemodel/1207v3plan19_correct_data/" + modelname

# define prerpocess_input
from keras.applications.xception import preprocess_input as preprocess_input_ception
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
if basemodel_name == 'resnet50':
    preprocess_input = preprocess_input_resnet
elif basemodel_name == 'xception' or 'inception_v3' or 'inception_resnet_v2':
    preprocess_input = preprocess_input_ception
else:
    print('basemodel_name not defined！')
    raise NameError

In [ ]:
train_batch_size = 56
valid_batch_size = 56
train_gen = MyImageDataGenerator(
    preprocessing_function=preprocess_input,
    width_shift_range=0.05,
    height_shift_range=0.05,
    rotation_range=6,
    shear_range=5,
    channel_shift_range=15,
    zoom_range=(.9, 1.1)
)

gen = MyImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_gen.myflow_from_directory(os.path.join(basedir, 'train'),  model_image_size,
                                                shuffle=True, batch_size=train_batch_size, my_horizontal_flip=True,randomHSV=False,myrescale=True)

valid_generator = gen.myflow_from_directory(os.path.join(basedir, 'valid'),  model_image_size,
                                          shuffle=True, batch_size=valid_batch_size)
train_samples_epoch = train_generator.samples
print("samples_train_epoch = {}".format(train_samples_epoch))

steps_train = len(train_generator)
print("steps_train = {}".format(steps_train))

valid_samples_epoch = valid_generator.samples
print("samples_valid_epoch = {}".format(valid_samples_epoch))

steps_valid = len(valid_generator)
print("steps_valid = {}".format(steps_valid))

In [ ]:
model = load_model(old_savemodel_path, custom_objects={'DepthwiseConv2D': DepthwiseConv2D})
multi_model = multi_gpu_model(model, gpus=4)

In [ ]:
class MyModelCheckpoint(Callback):

    def __init__(self, model, path, init_loss=None):
        self.mymodel = model
        self.path = path
        if init_loss == None:
            self.best_loss = np.inf
        else:
            self.best_loss = init_loss

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs['val_loss']
        val_acc = logs['val__count_accuracy']
        if val_loss < self.best_loss or val_acc>0.99:
            print("\nValidation loss decreased from {0:.5f} to {1:.5f}, saving model".format(self.best_loss, val_loss))
            savedmodel_name = '{}_{:.4f}_{:.4f}.h5'.format(self.path, val_loss, val_acc)
            self.mymodel.save(savedmodel_name, overwrite=True)
            self.best_loss = val_loss

In [ ]:
checkpoint = MyModelCheckpoint(model, savemodel_path)
multi_model.compile(optimizer=Adam(lr=1e-3,amsgrad=True), 
              loss=myloss(),
              metrics=[count_accuracy()])
multi_model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=100,
                   validation_data=valid_generator, validation_steps=steps_valid, 
                          max_queue_size=64, workers=12,
                 callbacks=[checkpoint, 
                EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')])

In [ ]:

multi_model.compile(optimizer=Adam(lr=1e-4,amsgrad=True), 
              loss=myloss(),
              metrics=[count_accuracy()])
multi_model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=100,
                   validation_data=valid_generator, validation_steps=steps_valid, 
                          max_queue_size=64, workers=12,
                 callbacks=[checkpoint, 
                EarlyStopping(monitor='val_loss', min_delta=0, patience=7, verbose=0, mode='min')])

In [ ]:
multi_model.compile(optimizer=Adam(lr=1e-5,amsgrad=True), 
              loss=myloss(),
              metrics=[count_accuracy()])
multi_model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=100,
                   validation_data=valid_generator, validation_steps=steps_valid, 
                          max_queue_size=64, workers=12,
                 callbacks=[checkpoint, 
                EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='min')])